# Information Retrieval project 2022

### Names
+ Mashra Marwan
+ Kiselov Nikita
+ Serré Gaëtan

## Imports

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy.matcher import PhraseMatcher
from spacy.util import filter_spans
from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span
from collections import Counter
from nltk.tokenize import MWETokenizer
from nltk.util import Trie
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')
tqdm.pandas()
spacy.__version__ 

[nltk_data] Downloading package wordnet to /Users/nkise/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/nkise/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


'3.2.4'

Our first goal here is to create a model capable of recognizing the term in our patents. For that, we will be using a Named Entity Recognition model given by `spacy`.

## Read and preprocess data

In [2]:
# if you've already unzipped the file
patent_data=open('G06K.txt').read().strip()

# split into patents texts | 1 entry = 1 patent
patent_texts = patent_data.split('\n\n')

# split each patent into lines
patent_lines = patent_data.split('\n')

## 👀 Extract features

In [3]:
# here are the potential terms
mwes = open('manyterms.lower.txt').read().lower().strip().split('\n')
print(mwes[44444:44456])
print(len(mwes),'mwes')

['antonio superchi', 'antonio tarver', 'antonio torres jurado', 'antonio valdes', 'antonio valdes y fernandez bazan', 'antonio valdez', 'antonio valdés y bazán', 'antonio valdés y fernández bazán', 'antonio valente', 'antonio vitali', 'antonio vivaldi', 'antonio xavier machado e cerveira']
743274 mwes


### We extract the terms from our patents using manyterms

In [4]:
# Here lowercase=False option is used to keep the original case of the terms, since we possibly could have term abbreviations. Like API, CAT, etc.
cvectorizer = CountVectorizer(ngram_range=(1, 4), stop_words="english", vocabulary=mwes, lowercase=True)
X=cvectorizer.fit_transform(patent_texts)

# Show top-25 most frequent terms
termdf_cv = pd.DataFrame(np.sum(X, axis=0), columns=cvectorizer.get_feature_names()).T.sort_values(by = 0, ascending = False)
termdf_cv.head(25)

/Users/nkise/miniforge3/envs/uni-env/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:1322: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
/Users/nkise/miniforge3/envs/uni-env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0
electronic device,16280
image processing,12224
control unit,9263
mobile terminal,9165
information processing,7732
neural network,6734
user interface,6177
computer readable,6103
fingerprint sensor,5980
display device,5666


## 🪄 SpaCy NER

Let's start from understanding. Here is an example of showing part of text on one patent with default NER.

In [5]:
nlp = spacy.load("en_core_web_lg")
doc = nlp(patent_texts[0][18000:20000]) # 
displacy.render(doc, style="ent", jupyter = True)

We want to create a such model capable of recognizing the terms that are in the context of our patents. For that, we need to create a dataset and we will be using `manyterms` as a terms database.

### Create DataSet

We need to create propper dataset that is compatible with SpaCy 3.0 to train a NER model.

In [6]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(text) for text in termdf_cv.index]
matcher.add("Tech", patterns)

In [7]:
train_lines, test_lines = train_test_split(patent_lines, test_size=0.3, random_state=42)

We are using PharsesMatcher to find entities similar to one from mayterms.txt  
Then Span is labeled and saved into the binary `.spacy` format

In [8]:
def create_dataset(text, n_lines, filename, offset=0):
  LABEL = "TECH"
  doc_bin = DocBin() # create a DocBin object

  for training_example  in tqdm(text[offset:offset+n_lines]):
      doc = nlp.make_doc(training_example) 
      ents = []
      
      for match_id, start, end in matcher(doc):
          span = Span(doc, start, end, label=LABEL)
          if span is None:
              print("Skipping entity")
          else:
              ents.append(span)

      filtered_ents = filter_spans(ents)
      doc.ents = filtered_ents 
      doc_bin.add(doc)
  doc_bin.to_disk(filename)

Uncomment to re-save the dataset:

In [9]:
# create_dataset(train_lines, 40_000, "training_data.spacy")
# create_dataset(test_lines, 12_000, "valid_data.spacy")

  0%|          | 0/40000 [00:00<?, ?it/s]

  0%|          | 0/12000 [00:00<?, ?it/s]

Now that our datasets are created, we can train a spacy NER model.

### Train the model

Donwnload __base_config.cfg__ for your system at https://spacy.io/usage/training#quickstart

In [10]:
# Run to generate full training config
# !python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Run the training. The best and last model will be stored into __./spacy_output__  

In [11]:
# !python -m spacy train config.cfg --output ./spacy_output --paths.train ./training_data.spacy --paths.dev ./valid_data.spacy --gpu-id 0

### 💻 Load pre-trained model

In [17]:
!git lfs install
!git clone https://huggingface.co/kinivi/ner_patent_g06k

/Users/nkise/Desktop/Paris-Saclay 🎓/Information retrieval/patent_ner_linking


### Test the model

In [ ]:
nlp = spacy.load("./ner_patent_g06k/spacy_output/model-best")

doc = nlp("Wi-Fi Direct (registered trademark, which will be hereinafter referred to as WFD) \
           corresponding to a technology for directly performing a communication based on a \
           wireless LAN between communication devices without intermediation of an access \
           point (hereinafter referred to as AP) is standardized in Wi-Fi Alliance serving \
           as a wireless LAN industry group.")

colors = {"TECH": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

In our patents, a lot of terms are abbreviated.
For example, `WI-FI Direct` is mentioned as `WFD`, or `P2P Group Owner` as `GO`.

Our original model does not recognize these abbreviations and therefore a huge part of the terms are ignored.
So we will fine-tune our model using prodigy

## 🦄 Prodigy

We create a train dataset for fine-tuning our ner model.

In [ ]:
# !prodigy ner.correct fine_tune_g06k2 spacy_output/model-best G06K.txt --loader txt --label TECH

<img src=img/annotations.png style="width: 40%; height:40%"></img>

Now let's fine-tune our model!

In [ ]:
# !prodigy train ./prodigy_output/ --ner fine_tune_g06k --base-model spacy_output/model-best --gpu-id 0

Now our refined model should recognize better the abbreviations.

In [13]:
nlp = spacy.load("./ner_patent_g06k/prodigy_output/model-best")

doc = nlp("According to the WFD, the communication is performed when one of the \
           communication devices that directly perform the wireless LAN communication \
           operates as the AP. According to the WFD, a role of the device that operates \
           as the AP will be referred to as P2P Group Owner (hereinafter, referred to as GO). \
           On the other hand, a role of the device that participates in a network generated by \
           the GO will be referred to as P2P Client (hereinafter, referred to as CL). \
           According to the WFD, a communication parameter necessary for participating in \
           the network generated by the GO is shared between the devices by transmitting the \
           communication parameter from the GO to the CL, and thereafter, the wireless \
           communication according to the WFD is executed on the basis of the shared communication parameter.")

colors = {"TECH": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

This fine tuned model is more accurate than the first one. We will use it in our Hearst patterns recognition.

## 🏆 Evaluation

#### Load model and data

In [25]:
# load best model
nlp_ner = spacy.load("./spacy_output/model-best")

# load test lines
with open('test_lines.txt', 'r') as f:
    test_lines = f.readlines()
f.close()

# read homonym_list.txt with pairs of homonyms
homonyms_df = pd.read_csv('./hearst_patterns/hearst_patterns.247.csv')
homonyms_df

,Unnamed: 0,word1,word2,relation,label,text
0,0,wireless communication,wireless LAN,rhyper,-1,104 denotes a wireless communication unit conf...
1,1,wireless communication,wireless LAN,rhyper,-1,204 denotes a wireless communication unit conf...
2,2,semiconductor memory,random access,rhyper,-1,In the case where the processing circuit 1801 ...
3,3,visible light,ultraviolet light,rhyper,-1,Fig. 1 shows the driver monitoring system 1. T...
4,4,visible light,infrared light,rhyper,-1,Fig. 1 shows the driver monitoring system 1. T...
...,...,...,...,...,...,...
242,242,digital image,the computer,rhyper,-1,Some example implementations provide a compute...
243,243,digital image,the computer,rhyper,-1,Clause 1. A method of testing a battery compri...
244,244,image sensor,complementary metal oxide semiconductor,rhyper,-1,The imaging unit 13 has an image sensor such a...
245,245,text processing,optical character recognition,rhyper,-1,Characters may be identified and extracted fro...


#### 🌐 Word-Net

Here we are evaluating extracted Hypernyms using WordNet. Here is an example how it works

In [26]:
cd_rom = wn.synsets('CD-ROM', pos='n')
computer = wn.synsets('computer', pos='n')

In [27]:
for synset in cd_rom:
    for synset2 in computer:
        print(synset, synset2)
        print("Score:", synset.wup_similarity(synset2))

Synset('cd-rom.n.01') Synset('computer.n.01')
Score: 0.7
Synset('cd-rom.n.01') Synset('calculator.n.01')
Score: 0.4


In [28]:
cd_rom[0].shortest_path_distance(computer[0])

6

Run on our list of hypernyms

In [29]:
def wordnet_distance(word1, word2):
    parent = wn.synsets(word1.replace(' ', '_'))
    subclass = wn.synsets(word2.replace(' ', '_'))
    scores = [0]
    try:
        for synset in parent:
            for synset2 in subclass:
                scores.append(synset.wup_similarity(synset2))
    except:
        return np.nan

    return np.round(max(scores), 2)

In [30]:
# iterate throw homonyms_df
results_wordnet = []
for index, row in tqdm(homonyms_df.iterrows()):
    if row["label"] == -1:
        parent = row['word1']
        subclass = row['word2']
    else:
        parent = row['word2']
        subclass = row['word1']
    res = wordnet_distance(parent, subclass)
    results_wordnet.append(res)
    print(parent, "⬅️", subclass, ": ", res)


0it [00:00, ?it/s]

wireless communication ⬅️ wireless LAN :  0
wireless communication ⬅️ wireless LAN :  0
semiconductor memory ⬅️ random access :  0
visible light ⬅️ ultraviolet light :  0.91
visible light ⬅️ infrared light :  0.91
mobile device ⬅️ mobile phone :  0
telecommunication network ⬅️ the internet :  0
reflective glass ⬅️ one-way mirror :  0
digital camera ⬅️ the system :  0
mass memory device ⬅️ hard drive :  0
virtual environment ⬅️ virtual environment :  0
two-dimensional code ⬅️ a square :  0
the image ⬅️ two-dimensional code :  0
geographical location data ⬅️ longitude and latitude :  0
geographical location data ⬅️ longitude and latitude :  0
information processing system ⬅️ electronic pen :  0
start signal ⬅️ the signal :  0
storage medium ⬅️ computer readable :  0.18
storage medium ⬅️ computer readable :  0.18
storage medium ⬅️ hard disk :  0.67
storage medium ⬅️ semiconductor memory :  0
storage medium ⬅️ hard disk :  0.67
storage medium ⬅️ semiconductor memory :  0
electronic device 

Save to the dataframe

In [31]:
# save to dataframe
homonyms_df['wordnet_distance'] = results_wordnet 

#### 🧪 Spacy embeddings
What if model already has links between words? Since it's trained on the corpus data, it should be able to find similarity between words.  
This is what can be useful while evaluationg our hyponyms list. We could run it and find low-similar elemts for further analysis.

In [32]:
word_1 = nlp("cloud platform")
word_2 = nlp("service provider")

print(word_1, "<->", word_2, word_1.similarity(word_2))

cloud platform <-> service provider 0.48239135894127


Run on our list of hypernyms

In [33]:
def spacy_score(word1, word2):
    word1 = nlp(word1)
    word2 = nlp(word2)
    return word1.similarity(word2)

In [34]:
# iterate throw homonyms_df
results_spacy = []
for index, row in tqdm(homonyms_df.iterrows()):
    if row["label"] == -1:
        parent = row['word1']
        subclass = row['word2']
    else:
        parent = row['word2']
        subclass = row['word1']

    res = spacy_score(parent, subclass)
    results_spacy.append(res)
    print(parent, "⬅️", subclass, ": ", res)


0it [00:00, ?it/s]

wireless communication ⬅️ wireless LAN :  0.7988240405990368
wireless communication ⬅️ wireless LAN :  0.7988240405990368
semiconductor memory ⬅️ random access :  0.4562830437539974
visible light ⬅️ ultraviolet light :  0.9213419926003001
visible light ⬅️ infrared light :  0.9114937304850219
mobile device ⬅️ mobile phone :  0.7980449644673413
telecommunication network ⬅️ the internet :  0.3566173245450127
reflective glass ⬅️ one-way mirror :  0.43850586242637135
digital camera ⬅️ the system :  0.29880282444533746
mass memory device ⬅️ hard drive :  0.668589128614972
virtual environment ⬅️ virtual environment :  1.0
two-dimensional code ⬅️ a square :  0.030406911246921867
the image ⬅️ two-dimensional code :  -0.28950677451373197
geographical location data ⬅️ longitude and latitude :  0.2530226461434113


/Users/nkise/miniforge3/envs/uni-env/lib/python3.9/site-packages/numpy/core/_methods.py:235: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


geographical location data ⬅️ longitude and latitude :  0.2530226461434113
information processing system ⬅️ electronic pen :  0.30134069311937467
start signal ⬅️ the signal :  0.5420448808237851
storage medium ⬅️ computer readable :  0.6335364128335599
storage medium ⬅️ computer readable :  0.6335364128335599
storage medium ⬅️ hard disk :  0.5238650175071684
storage medium ⬅️ semiconductor memory :  0.6051938531665448
storage medium ⬅️ hard disk :  0.5238650175071684
storage medium ⬅️ semiconductor memory :  0.6051938531665448
electronic device ⬅️ mobile phone :  0.7084252507788912
neural network ⬅️ convolutional neural network :  0.9166362725093549
communication network ⬅️ the Internet :  0.39472354731538645
communication network ⬅️ the Internet :  0.39472354731538645
semiconductor memory ⬅️ flash memory :  0.8473547281050336
optical media ⬅️ optical discs :  0.6129062358351681
magnetic media ⬅️ hard disks :  0.4780298046805386
communication network ⬅️ cellular network :  0.7751448356

Add results to the dataframe

In [35]:
homonyms_df['spacy_distance'] = results_spacy

In [37]:
homonyms_df.head()

,Unnamed: 0,word1,word2,relation,label,text,wordnet_distance,spacy_distance
0,0,wireless communication,wireless LAN,rhyper,-1,104 denotes a wireless communication unit conf...,0.00,0.798824
1,1,wireless communication,wireless LAN,rhyper,-1,204 denotes a wireless communication unit conf...,0.00,0.798824
2,2,semiconductor memory,random access,rhyper,-1,In the case where the processing circuit 1801 ...,0.00,0.456283
3,3,visible light,ultraviolet light,rhyper,-1,Fig. 1 shows the driver monitoring system 1. T...,0.91,0.921342
4,4,visible light,infrared light,rhyper,-1,Fig. 1 shows the driver monitoring system 1. T...,0.91,0.911494


#### 📜 Wikidata

In [38]:
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from qwikidata.sparql import (get_subclasses_of_item,
                              return_sparql_query_results)
import wptools

In [39]:
parent_name = "computer"
candidate_name = "iPad"

# get Wikidata item for parent
page = wptools.page(parent_name)
data = page.get_parse(show=False)
q_parent_class = data.data['wikibase']
q_parent_class

en.wikipedia.org (parse) computer


'Q68'

In [40]:
# use convenience function to get subclasses of an item as a list of item ids
subclasses_list = get_subclasses_of_item(q_parent_class)
len(subclasses_list)

3452

In [41]:
#print some of this classes
for subclass in subclasses_list[:5]:
    q42_dict = get_entity_dict_from_api(subclass)
    print(WikidataItem(q42_dict).get_label())

computer
personal computer
IBM 704
microcomputer
analog computer


In [42]:
page = wptools.page(candidate_name)
data = page.get_parse(show=False)
data.data['wikibase']

en.wikipedia.org (parse) iPad


'Q2796'

In [43]:
print(f"Is class `{parent_name}` is a subclass of `{candidate_name}`: ", data.data['wikibase'] in subclasses_list)

Is class `computer` is a subclass of `iPad`:  True


Run on our list of hypernyms

In [44]:
def wikidata_is_subclass(word_1, word_2):
    parent_name = word_1
    candidate_name = word_2

    # get Wikidata item for parent
    page = wptools.page(parent_name)
    try:
        data_parent = page.get_parse(show=False)
    except:
        print(f"Could not find Wikidata item for `{parent_name}`")
        return "⚠️"
    q_parent_id = data_parent.data['wikibase']

    # get Wikidata item for candidate
    page = wptools.page(candidate_name)
    try:
        data_subclass = page.get_parse(show=False)
    except:
        print(f"Could not find Wikidata item for `{candidate_name}`")
        return "⚠️"
    q_subclass_id = data_subclass.data['wikibase']

    # use convenience function to get subclasses of an item as a list of item ids
    subclasses_list = get_subclasses_of_item(q_parent_id)

    res = q_subclass_id in subclasses_list
    if res:
        return "✅"
    else:
        return "❌"

In [45]:
# iterate throw homonyms_df
wikidata_results = []
for index, row in tqdm(homonyms_df.iterrows()):
    if row["label"] == -1:
        parent = row['word1']
        subclass = row['word2']
    else:
        parent = row['word2']
        subclass = row['word1']

    res = wikidata_is_subclass(parent, subclass)
    wikidata_results.append(res)
    print(parent, "⬅️", subclass, ": ",wikidata_is_subclass(parent, subclass))


0it [00:00, ?it/s]

en.wikipedia.org (parse) wireless communication
en.wikipedia.org (parse) wireless LAN
en.wikipedia.org (parse) wireless communication
en.wikipedia.org (parse) wireless LAN


wireless communication ⬅️ wireless LAN :  ✅


en.wikipedia.org (parse) wireless communication
en.wikipedia.org (parse) wireless LAN
en.wikipedia.org (parse) wireless communication
en.wikipedia.org (parse) wireless LAN


wireless communication ⬅️ wireless LAN :  ✅


...

image sensor ⬅️ complementary metal oxide semiconductor :  ❌


en.wikipedia.org (parse) text processing
en.wikipedia.org (parse) optical character recognition
en.wikipedia.org (parse) text processing
en.wikipedia.org (parse) optical character recognition


text processing ⬅️ optical character recognition :  ❌


en.wikipedia.org (parse) text field
en.wikipedia.org (parse) text field
en.wikipedia.org (parse) text field
en.wikipedia.org (parse) text field


text field ⬅️ text field :  ✅


Add results to the dataframe

In [ ]:
homonyms_df['wikidata_is_subclass'] = wikidata_results

In [ ]:
homonyms_df

#### Process table

In [ ]:
# create columns wordnet_is_subclass, fill with "⚠️" if not 0 or np.nan, otherwise fill with "✅" if wordnet_distance > 0.7, otherwise fill with "❌"
homonyms_df['wordnet_is_subclass'] = homonyms_df['wordnet_distance'].apply(lambda x: "⚠️" if x == 0 or np.isnan(x) else "✅" if x > 0.7 else "❌")

# do the same for spacy
homonyms_df['spacy_is_subclass'] = homonyms_df['spacy_distance'].apply(lambda x: "⚠️" if x == 0 or np.isnan(x) else "✅" if x > 0.3 else "❌")


homonyms_df

In [ ]:
homonyms_df[[ "label", "word1", "word2", "wordnet_is_subclass", "spacy_is_subclass", "wikidata_is_subclass"]].to_csv("./homonyms_results.csv", index=False)

In [ ]:
homonyms_df[["label", "word1", "word2",  "wordnet_distance", "spacy_distance", "wordnet_is_subclass", "spacy_is_subclass", "wikidata_is_subclass"]].to_csv("./homonyms_results_detailed.csv", index=False)